In [ ]:
import hist
import boost_histogram as bh
import numpy as np
import mplhep as hep
import matplotlib.pyplot as plt
from utilities import boostHistHelpers as hh
from wremnants import histselections as sel
from wremnants import datasets2016
from wremnants import plot_tools
import wremnants
import h5py
import uproot
import narf
hep.style.use(hep.style.ROOT)

In [ ]:
with h5py.File("../mw_with_mu_eta_pt.hdf5", "r") as f:
    result = narf.ioutils.pickle_load_h5py(f["results"])
    print(result['WminustaunuPostVFP']['event_count'])
    print(result.keys())


In [ ]:
f = h5py.File("../mw_with_mu_eta_pt_maxFiles10.hdf5", "r")
results = narf.ioutils.pickle_load_h5py(f["results"])
print(results['WplusmunuPostVFP']['output'].keys())
sw = results['WplusmunuPostVFP']['output']['muonScaleSyst_responseWeights_gensmear'].get()
nom = results['WplusmunuPostVFP']['output']['nominal_gen_smeared'].get()
action = lambda x: x.project('pt')

fig = plot_tools.makePlotWithRatioToRef(
    [
        action(nom),
        action(sw[...,143,0]),
        action(sw[...,143,1])
    ],
    labels = [
        'nominal', 
        'smearing weights DOWN',
        'smearing weights UP'
    ],
    colors = ['black', 'green', 'green'],
    rrange = [0.99995, 1.00005],
    nlegcols = 1,
    rlabel = 'var/nominal',
    xlabel = r'GEN smeared $p_T^{\ell}$',
    grid = True
)
print(action(sw[...,143,0]).values())

In [ ]:
wp = boost['WplusmunuPostVFP']['output']
wm = boost['WminusmunuPostVFP']['output']
print(wp['muonScaleSyst_responseWeights_gensmear'].axes)

In [ ]:
action = lambda x: x.project('pt')
#action = lambda x: sel.unrolledHist(x)

nominal_gensmear = action(wp['nominal_gen_smeared'][{'charge':1j}])
ms_from_massweights = [action(wp['nominal_muonScaleSyst_gen_smear'][{'charge':1j}][...,0,0]), 
                       action(wp['nominal_muonScaleSyst_gen_smear'][{'charge':1j}][...,1,0])]
ms_from_smearing = [action(wp['muonScaleSyst_responseWeights_gensmear'][{'charge':1j}][...,143,0]), 
                    action(wp['muonScaleSyst_responseWeights_gensmear'][{'charge':1j}][...,143,1])]
ms_from_scale_var = [action(wp['nominal_muonScaleVariationDnTenthmil_gen_smear'][{'charge':1j}]),
                     action(wp['nominal_muonScaleVariationUpTenthmil_gen_smear'][{'charge':1j}])]
fig = plot_tools.makePlotWithRatioToRef(
    [
        nominal_gensmear,
        ms_from_massweights[0],
        ms_from_massweights[1],
        ms_from_smearing[0],
        ms_from_smearing[1],
        ms_from_scale_var[0],
        ms_from_scale_var[1]#* nominal.sum().value / ms_from_scale_var.sum().value
    ],
    labels = [
        'nominal_gensmear', 
        'massweights DOWN',
        'massweights UP',
        'smearing weights DOWN',
        'smearing weights UP',
        'manual scale shift DOWN',
        'manual scale shift UP'
    ],
    colors = ['black', 'green', 'green', 'red', 'red', 'orange', 'orange'],
    rrange = [0.9999, 1.0001],
    nlegcols = 1,
    rlabel = 'var/nominal',
    xlabel = r'smeared GEN $p_T^{\ell}$',
    grid = True
)


fig = plot_tools.makePlotWithRatioToRef(
    [
        ms_from_scale_var[0],
        ms_from_massweights[0],
        ms_from_smearing[0] #* ms_from_massweights[0].sum().value/ ms_from_smearing[0].sum().value,
        
    ],
    labels = [ 
        'manual scale shift DOWN',
        'massweights DOWN',
        'smearing weights DOWN'
        
    ],
    colors = ['orange','green', 'red'],
    rrange = [0.9999, 1.0001],
    nlegcols = 1,
    rlabel = 'var/massweights',
    xlabel = r'$p_T^{\ell}$',
    grid = True
)

fig = plot_tools.makePlotWithRatioToRef(
    [
        ms_from_scale_var[1],
        ms_from_massweights[1],
        ms_from_smearing[1] #* ms_from_massweights[0].sum().value/ ms_from_smearing[0].sum().value,
        
    ],
    labels = [ 
        'manual scale shift UP',
        'massweights UP',
        'smearing weights UP'
        
    ],
    colors = ['orange','green', 'red'],
    rrange = [0.999, 1.001],
    nlegcols = 1,
    rlabel = 'var/massweights',
    xlabel = r'$p_T^{\ell}$',
    grid = True
)

In [ ]:
ms_from_smearing[0].values()

In [ ]:
action = lambda x: x.project('pt')
#action = lambda x: sel.unrolledHist(x)

nominal = action(wp['nominal'][{'charge':1j}])
ms_from_smearing = [action(wp['nominal_muonScaleSyst_responseWeights'][{'charge':1j}][...,143,0]), 
                    action(wp['nominal_muonScaleSyst_responseWeights'][{'charge':1j}][...,143,1])]


fig = plot_tools.makePlotWithRatioToRef(
    [
        nominal,
        ms_from_smearing[0],
        ms_from_smearing[1]
    ],
    labels = [
        'nominal', 
        r'J/$\Psi$ massfit variation #72 DOWN',
        r'J/$\Psi$ massfit variation #72 UP',
    ],
    colors = ['black', 'red', 'blue'],
    rrange = [0.99995, 1.00005],
    nlegcols = 1,
    rlabel = 'var/nominal',
    xlabel = r'RECO $p_T^{\ell}$',
    grid = True
)

In [ ]:
import uproot
import hist
import numpy as np
from wremnants import muon_validation

n_scale_params = 3
n_tot_params = 4
n_eta_bins = 24
filepath = "../wremnants/data/calibration/calibrationJDATA_smeared_v718.root"

f = uproot.open(filepath)
cov = f['covariance_matrix'].to_hist()
cov_scale_params = muon_validation.get_jpsi_scale_param_cov_mat(cov, n_scale_params, n_tot_params, n_eta_bins)

w,v = np.linalg.eigh(cov_scale_params)
var_mat = np.sqrt(w) * v
axis_eta = hist.axis.Regular(n_eta_bins, -2.4, 2.4, name = 'eta')
axis_scale_params = hist.axis.Regular(n_scale_params, 0, 1, name = 'scale_params')
axis_scale_params_unc = hist.axis.Regular(
    n_eta_bins * n_scale_params, 0, n_eta_bins * n_scale_params,
    underflow = False, overflow = False,  name = 'unc'
)
hist_scale_params_unc = hist.Hist(axis_eta, axis_scale_params, axis_scale_params_unc)
for i in range(n_eta_bins):
    lb, ub = i * n_scale_params, (i + 1) * n_scale_params
    hist_scale_params_unc.view()[i,...] = var_mat[lb:ub][:]



In [ ]:
print(f.keys())
f['correlation_matrix'].values()
cov.values().diagonal()

In [ ]:
wp['nominal_muonScaleSyst_responseWeights']

In [ ]:
hep.hist2dplot(cov)

In [ ]:
hep.hist2dplot(hist_scale_params_unc[{'scale_params':0}])

In [ ]:
hep.hist2dplot(hist_scale_params_unc[{'scale_params':1}])

In [ ]:
hep.hist2dplot(hist_scale_params_unc[{'scale_params':2}])

In [ ]:
f = uproot.open('../wremnants/data/calibration/calibrationJDATA_smeared_v718.root')

#print("cov mat of A, e, M, res in eta bins", f['covariance_matrix'].values())
A_uncs = []
e_uncs = []
M_uncs = []
As = f['A'].values()
es = f['e'].values()
Ms = f['M'].values()
for i in range(24):
    A_uncs.append(cov_scale_params.diagonal()[3*i])
    e_uncs.append(cov_scale_params.diagonal()[3*i+1])
    M_uncs.append(cov_scale_params.diagonal()[3*i+2])
print("uncs on A: \n")
for A_unc, A in zip(A_uncs,As): print(np.sqrt(A_unc))
print('\n')
print("uncs on e: \n")
for e_unc, e in zip(e_uncs, es): print(np.sqrt(e_unc))
print('\n')
print("uncs on M: \n")
for M_unc, M in zip(M_uncs, Ms): print(np.sqrt(M_unc))

In [ ]:
swv = wp['nominal_muonScaleSyst_responseWeights']

for i in range(swv[0,0,0,0,0,...,0].size):
    var_h_dn = hh.addHists(swv[{'charge':1j, 'downUpVar':0, 'unc':i}].project('pt'), -1 * nominal)
    var_h_up = hh.addHists(swv[{'charge':1j, 'downUpVar':1, 'unc':i}].project('pt'), -1 * nominal)
    varsqd_h_dn = hh.multiplyHists(var_h_dn, var_h_dn)
    varsqd_h_up = hh.multiplyHists(var_h_up, var_h_up)
    #print('varsqd dn/up in bin 16 is:',  varsqd_h_dn[15].value/varsqd_h_up[15].value, 'in variable #', i)
    if i == 0:
        tot_varsqd_h_dn = varsqd_h_dn 
        tot_varsqd_h_up = varsqd_h_up
    else: 
        tot_varsqd_h_dn = hh.addHists(tot_varsqd_h_dn, varsqd_h_dn)
        tot_varsqd_h_up = hh.addHists(tot_varsqd_h_up, varsqd_h_up)
tot_var_h_dn = hh.sqrtHist(tot_varsqd_h_dn)
tot_var_h_up = hh.sqrtHist(tot_varsqd_h_up)

h_dn = hh.addHists(nominal, -1 * tot_var_h_dn)
h_up = hh.addHists(nominal, tot_var_h_up)

fig = plot_tools.makePlotWithRatioToRef(
    [
        nominal,
        h_dn,
        h_up
    ],
    labels = [
        'nominal', 
        r'$J/\Psi$ massfit stat. unc. band DOWN',
        r'$J/\Psi$ massfit stat. unc. band UP',
    ],
    colors = ['black','red', 'blue'],
    rrange = [0.9995, 1.0005],
    nlegcols = 1,
    rlabel = 'var/nominal',
    xlabel = r'RECO $p_T^{\ell}$',
    grid = True
)    

In [ ]:
fig, ax = plt.subplots()
hep.histplot(wp['smearing_weights_up'].project('smearing_weight'), label = "up", ax=ax)
hep.histplot(wp['smearing_weights_down'].project('smearing_weight'), label = "down", ax=ax)
ax.legend()
ax.set_xlim(0.99,1.01)
ax.grid(which='both')

In [ ]:
wp_label = r'$W^+ \rightarrow \mu^+ \nu$' 
fig = plot_tools.makePlotWithRatioToRef(
    [
        wp['gen_smeared_over_gen'].project('reco_pt_over_gen'),
        wp['uncrct_over_gen'].project('reco_pt_over_gen'),
        wp['cvhbs_over_gen'].project('reco_pt_over_gen'),
        wp['crctd_over_gen'].project('reco_pt_over_gen')
    ],
    labels = [ 
        'gen smeared / gen muons',
        'uncorrected muons / gen muons',
        'cvhbs muons / gen muons',
        'corrected muons / gen muons'
    ],
    colors = ['black','red', 'orange', 'green'],
    rrange = [0.85, 1.15],
    nlegcols = 1,
    rlabel = 'corrected/uncorrected',
    xlabel = r'RECO/GEN $p_T^{\mu}$',
    #xlim = [0.9995, 1.0005],
    grid = True,
    bin_density = 400,
    x_ticks_ndp = 3,
    plot_title = wp_label
)

In [ ]:
wm_label = r'$W^- \rightarrow \mu^- \bar{\nu}$' 
fig = plot_tools.makePlotWithRatioToRef(
    [
        wm['gen_smeared_over_gen'].project('reco_pt_over_gen'),
        wm['uncrct_over_gen'].project('reco_pt_over_gen'),
        wm['cvhbs_over_gen'].project('reco_pt_over_gen'),
        wm['crctd_over_gen'].project('reco_pt_over_gen')
    ],
    labels = [ 
        'gen smeared / gen muons',
        'uncorrected muons / gen muons',
        'cvhbs muons / gen muons',
        'corrected muons / gen muons'
    ],
    colors = ['black','red', 'orange', 'green'],
    rrange = [0.85, 1.15],
    nlegcols = 1,
    rlabel = 'corrected/uncorrected',
    xlabel = r'RECO/GEN $p_T^{\mu}$',
    #xlim = [0.9995, 1.0005],
    grid = True,
    bin_density = 400,
    x_ticks_ndp = 3,
    plot_title = wm_label
)

In [ ]:
nominal_gen_smearing = wp['nominal_gen_smeared'][{'charge':1j}].project('pt', 'eta')
print(nominal_gen_smearing)

gen_smeared_msv_from_sw = [wp['muonScaleSyst_responseWeights_gensmear'][{'charge':1j}][...,0,0].project('pt', 'eta'), 
                           wp['muonScaleSyst_responseWeights_gensmear'][{'charge':1j}][...,0,1].project('pt', 'eta')]
smearing_weights_per_bin_up = hh.divideHists(gen_smeared_msv_from_sw[0], nominal_gen_smearing)
smearing_weights_per_bin_dn = hh.divideHists(gen_smeared_msv_from_sw[1], nominal_gen_smearing)

nominal_reco = wp['nominal'][{'charge':1j}].project('pt', 'eta')
msv_reco_massweights = [wp['muonScaleSyst'][{'charge':1j}][...,0,0].project('pt', 'eta'), 
                        wp['muonScaleSyst'][{'charge':1j}][...,1,0].project('pt', 'eta')]
msv_reco_manual = [wp['muonScaleVariationDnTenthmil'][{'charge':1j}].project('pt', 'eta'), 
                   wp['muonScaleVariationUpTenthmil'][{'charge':1j}].project('pt', 'eta')]
#msv_reco_sw = [hh.multiplyHists(nominal_reco, smearing_weights_per_bin_dn),
#               hh.multiplyHists(nominal_reco, smearing_weights_per_bin_up)]

msv_reco_sw = [wp['muonScaleSyst_responseWeights'][...,0], wp['muonScaleSyst_responseWeights'][...,1]]
fig = plot_tools.makePlotWithRatioToRef(
    [
        action(nominal_reco),
        action(msv_reco_massweights[0]),
        action(msv_reco_massweights[1]),
        action(msv_reco_sw[0]),
        action(msv_reco_sw[1]),
        action(msv_reco_manual[0]),
        action(msv_reco_manual[1])#* nominal.sum().value / ms_from_scale_var.sum().value
    ],
    labels = [
        'nominal', 
        'massweights DOWN',
        'massweights UP',
        'smearing weights DOWN',
        'smearing weights UP',
        'manual scale shift DOWN',
        'manual scale shift UP'
    ],
    colors = ['black', 'green', 'green', 'red','red', 'orange', 'orange'],
    rrange = [0.999, 1.001],
    nlegcols = 1,
    rlabel = 'var/nominal',
    xlabel = r'RECO $p_T^{\ell}$',
    grid = True
)

In [ ]:
proj_axes = ['pt', 'eta']
a = wp['muonScaleSyst_gen_smear'].project(*proj_axes)
print(a.axes)
a.project('pt')
#print(wp['muonScaleSyst_gen_smear'][{'pt':hist.loc(26)}].project('pt'))

In [ ]:
proc = 'WplusmunuPostVFP'
print(any([x in proc for x in ['mlus', 'Plus']]))

In [ ]:
wp['muonScaleSyst_responseWeights']